In [1]:
# This file has other stuff, because I thought I'd be starting one-heating in this notebook... but 
# it turned out to be extra preprocessing instead!

import pickle
import pandas as pd
import numpy as np
import gzip
import datetime
from itertools import product
from scipy import interpolate ## For other interpolation functions.
import time

import sklearn.metrics

from sklearn.cross_validation import LabelKFold

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with gzip.open('merged7.pkl.gz', 'rb') as fd:
    data = pickle.load(fd)

In [7]:
data[data.activity_id == 'act2_3889418']

,activity_category,activity_id,adate,outcome,achar_1,achar_10,achar_2,achar_3,achar_4,achar_5,...,pchar_4,pchar_5,pchar_6,pchar_7,pchar_8,pchar_9,outcome_filled,outcome_filled_nona,outcome_leak,outcome_leak_int
1372164,5,act2_3889418,2022-10-20,1.0,-1,2251,-1,-1,-1,-1,...,24,8,2,10,3,3,1.0,1.0,1.0,1.0


In [6]:
with gzip.open('cvleak7-10fold.pkl.gz', 'rb') as fd:
    cvleak = pickle.load(fd)
#cvleak = pickle.load(open('cvleak7-10fold.pkl.gz', 'rb'))

data = pd.merge(data, cvleak, on='activity_id', how='left')

In [8]:
data.keys()

Index(['activity_category', 'activity_id', 'adate', 'outcome', 'achar_1',
       'achar_10', 'achar_2', 'achar_3', 'achar_4', 'achar_5', 'achar_6',
       'achar_7', 'achar_8', 'achar_9', 'people_id', 'pdate', 'pchar_10',
       'pchar_11', 'pchar_12', 'pchar_13', 'pchar_14', 'pchar_15', 'pchar_16',
       'pchar_17', 'pchar_18', 'pchar_19', 'pchar_20', 'pchar_21', 'pchar_22',
       'pchar_23', 'pchar_24', 'pchar_25', 'pchar_26', 'pchar_27', 'pchar_28',
       'pchar_29', 'pchar_30', 'pchar_31', 'pchar_32', 'pchar_33', 'pchar_34',
       'pchar_35', 'pchar_36', 'pchar_37', 'pchar_38', 'group_1', 'pchar_1',
       'pchar_2', 'pchar_3', 'pchar_4', 'pchar_5', 'pchar_6', 'pchar_7',
       'pchar_8', 'pchar_9', 'outcome_filled', 'outcome_filled_nona',
       'outcome_leak', 'outcome_leak_int'],
      dtype='object')

In [9]:
def split_traintest():
    testset = np.where(data['outcome'].isnull())
    trainset = np.where(~data['outcome'].isnull())

    return trainset, testset, data.iloc[trainset], data.iloc[testset]

trainset, testset, train, test = split_traintest()

In [10]:
data.outcome_filled_nona.isnull().sum()

0

In [11]:
# compute errors of leak backfill (useful references for upcoming NN tests)
print(sklearn.metrics.roc_auc_score(train.outcome.values, train.outcome_filled_nona.values)) # AUC

print(sklearn.metrics.mean_squared_error(train.outcome.values, train.outcome_filled_nona.values)) # MSE

print(np.sqrt(sklearn.metrics.mean_squared_error(train.outcome.values, train.outcome_filled_nona.values))) # RMSE

0.984487762204
0.0451716364123
0.21253620024


In [12]:
# Just for fun, compute AUC of a 10-fold leak interpolation using 4-folds
# Looks like the second fold consistently has outliers... so fold 1 should probably be the one worked on first

#for i in range(len(cv_val)):
#    print(sklearn.metrics.roc_auc_score(cv_val_tgt[i].outcome.values, cv_val[i].outcome_filled.values))


In [13]:
fold = 1

In [14]:
onehot_values = {}

for k in data.keys():
    if k == 'pchar_38': # only continuous *char
        continue
        
    if k == 'achar_10': # we're reducing this
        continue
        
    if 'char' in k:
        onehot_values[k] = sorted(data[k].unique())

In [15]:
tot = 0
for k in onehot_values.keys():
    tot += len(onehot_values[k])
    print(k, len(onehot_values[k]))
    
print(tot)

pchar_7 25
pchar_6 7
achar_4 8
pchar_1 2
pchar_36 2
pchar_8 8
pchar_29 2
pchar_10 2
pchar_32 2
achar_3 12
pchar_28 2
pchar_12 2
pchar_25 2
pchar_11 2
pchar_24 2
pchar_14 2
pchar_15 2
pchar_13 2
pchar_17 2
pchar_20 2
pchar_5 9
pchar_30 2
pchar_3 43
achar_9 20
pchar_16 2
achar_2 33
pchar_33 2
pchar_21 2
pchar_18 2
pchar_4 25
achar_5 8
pchar_23 2
pchar_22 2
pchar_34 2
achar_1 52
pchar_19 2
pchar_27 2
pchar_35 2
pchar_2 3
pchar_9 9
achar_6 6
achar_7 9
pchar_31 2
achar_8 19
pchar_26 2
pchar_37 2
354


In [16]:
# achar_10 has 6970 possible values!  figure out which ones are siginificant 

In [17]:
max(train.achar_10.value_counts().values), len(train.achar_10.value_counts())

(904683, 6516)

In [18]:
divs = [-1, 1, 4, 16, 40, 100, 200, 500, 1000, 2000, 4000, 8000, 20000, 904684]

vc = train.achar_10.value_counts().to_frame().reset_index().rename(columns={'index': 'value','achar_10':'counted'})

achar10_lowcounts = {}
for i in range(1, len(divs)):
    achar10_lowcounts[divs[i]] = vc[np.logical_and(vc['counted'].values > divs[i-1], vc['counted'].values <= divs[i])]

In [19]:
c = 0
for k in sorted(achar10_lowcounts.keys()):
    c += len(achar10_lowcounts[k])
    print(k, len(achar10_lowcounts[k]), c)

1 783 783
4 835 1618
16 1907 3525
40 1115 4640
100 818 5458
200 466 5924
500 347 6271
1000 128 6399
2000 57 6456
4000 32 6488
8000 10 6498
20000 11 6509
904684 7 6516


In [20]:
tgt_mean = np.mean(train.outcome.values)
std_mean = np.std(train.outcome.values)
mindiff3 = np.std(train.outcome.values) ** 3.0
mindiff4 = np.std(train.outcome.values) ** 4.0
print(tgt_mean, mindiff3, mindiff4)

0.443954396573 0.122651583249 0.0609393117382


In [21]:
# Build discard list:

discardsets = {}

discard_sizes = [1, 4, 16, 40]

for i in discard_sizes:
    discardsets[i] = achar10_lowcounts[i]['value'].values
    

In [22]:
# find test values that do not exist in train set (use on ALL types???)

train_values = sorted(train['achar_10'].unique())

discard_testonly = []
for i in sorted(test['achar_10'].unique()):
    if i not in train_values:
        discard_testonly.append(i)

discardsets[1] = np.concatenate([discardsets[1], np.array(discard_testonly)])

In [23]:
def build_discardset(b, mindiff):
    discard = []
    for i in achar10_lowcounts[b].iterrows():
        ratio = np.sum(train[train.achar_10 == i[1].value].outcome) / i[1].counted
        #print(i[1].value, i[1].counted, ratio - tgt_mean)
    
        if abs(ratio - tgt_mean) < mindiff:
            discard.append(i[1]['value'])
        
    return np.array(discard)

discardsets[100] = build_discardset(100, std_mean ** 2)
discardsets[200] = build_discardset(200, std_mean ** 2.5)
discardsets[400] = build_discardset(500, std_mean ** 3.0)
discardsets[1000] = build_discardset(1000, std_mean ** 4.0)


In [24]:
achar10_reduced = data.achar_10.values.copy()

for k in sorted(discardsets.keys(), reverse=True):
    print(k, len(discardsets[k]))
    
    mask = np.full(len(data), False, dtype=np.bool)
    
    for i in discardsets[k]:
        mask = np.logical_or(mask, data.achar_10 == i)
        
    achar10_reduced[np.where(mask)] = -k

1000 23
400 160
200 245
100 499
40 1115
16 1907
4 835
1 1237


In [25]:
len(np.unique(achar10_reduced))

956

In [26]:
data['achar_10_reduced'] = achar10_reduced
onehot_values['achar_10_reduced'] = sorted(data['achar_10_reduced'].unique())

tot = 0
for k in onehot_values.keys():
    tot += len(onehot_values[k])
#    print(k, len(onehot_values[k]))
    
print(tot)

1310


In [27]:
# before (re)building train/test sets, add day of week!

data['adate_dayofweek'] = np.array(list(map(lambda x: x.dayofweek, data.adate))) # SLOW!  move out somehow?
onehot_values['adate_dayofweek'] = sorted(data['adate_dayofweek'].unique())

In [28]:
if True: # Wasteful to recompute a constant every time
    mindate = pd.Timestamp('2022-07-17 00:00:00')
    maxdate = pd.Timestamp('2023-08-31 00:00:00')
    minpdate = pd.Timestamp('2020-05-18 00:00:00')
else:
    mindate = min(data['date'])
    maxdate = max(data['date'])
    minpdate = min(data['pdate'])
    
data['adate_daynum'] = (data.adate - mindate).values.astype('timedelta64[D]')  # fast enough to run anywhere

minpdate = pd.Timestamp('2020-05-18 00:00:00')
data['pdate_daynum'] = (data.pdate - mindate).values.astype('timedelta64[D]')

data['business_days_delta'] = np.busday_count(data.pdate.values.astype('datetime64[D]'), 
                                              data.adate.values.astype('datetime64[D]'))

In [29]:
data['adate_daynum'] = data.adate_daynum.astype('<m8[D]').values.astype(float)
data['pdate_daynum'] = data.pdate_daynum.astype('<m8[D]').values.astype(float)

In [30]:
cols = data.columns.copy()
cols = cols.drop('activity_id')
data_dups = data.duplicated(subset=cols)

data_dedup = data[~data_dups]
#train_cut_dups = train_cut[train_dups]

In [31]:
# date gap between events

dgap = {}

tmp_data = data[['group_1', 'adate_daynum', 'outcome_filled']]

#tmp_data_dedup2 = tmp_data_dedup[tmp_data_dedup.duplicated()]

groupdays = {}
remap = []

for g in tmp_data.groupby(['group_1']):
    sdays = np.array(sorted(g[1].adate_daynum.unique()))
    if len(sdays) == 1:
        continue
        
    sdays_mean = np.array([g[1][g[1].adate_daynum == d].outcome_filled.mean() for d in sdays])
    
    om_prevday = np.concatenate([[np.nan], sdays_mean[:-1]])
    om_nextday = np.concatenate([sdays_mean[1:], [np.nan]])
        
    diffdays = np.concatenate([[np.nan], np.diff(sdays)])

    c = 0
    for x in zip(sdays, diffdays, om_prevday, om_nextday):
        #remap[(g[0], x[0])] = x[1]
        c += 1
        remap.append((g[0], x[0], x[1], x[2], x[3]))

    '''
    if np.sum(sdays_mean == sdays_mean) != 0:
        print(sdays)
        print(sdays_mean)
        print(diffdays)
        print(om_prevday)
        print(om_nextday)
        break
    '''


In [32]:
df_remap = pd.DataFrame(remap)

df_remap.columns = ['group_1', 'adate_daynum', 'adate_gap', 'outcome_filled_prevday', 'outcome_filled_nextday']

subkeys = ['group_1', 'group_1', 'adate_daynum', 'activity_id']

data = pd.merge(data, df_remap, on=['group_1', 'adate_daynum'], how='left')

In [33]:
# number of people/group

ppg = []
for g in data.groupby('group_1'):
    ppg.append([g[0], len(g[1].people_id.unique())])

df_ppg = pd.DataFrame(ppg)
df_ppg.columns = ['group_1', 'people_per_group']

data = pd.merge(data, df_ppg, on='group_1', how='left')

In [34]:
# need to use dedup'd data here for events/groups+days, since it's per-day and we don't want it distorted

cols = data.columns.copy()
cols = cols.drop('activity_id')
data_dups = data.duplicated(subset=cols)

data_dedup = data[~data_dups]

In [35]:
ppg_d = []
for g in data_dedup.groupby(['group_1', 'adate_daynum']):
    ppg_d.append([g[0][0], g[0][1], len(g[1].people_id.unique()), len(g[1])])

df_ppg_d = pd.DataFrame(ppg_d)

df_ppg_d.columns = ['group_1', 'adate_daynum', 'people_per_group_adate', 'events_per_group_adate']

data = pd.merge(data, df_ppg_d, on=['group_1', 'adate_daynum'], how='left')

In [36]:
data.keys()

Index(['activity_category', 'activity_id', 'adate', 'outcome', 'achar_1',
       'achar_10', 'achar_2', 'achar_3', 'achar_4', 'achar_5', 'achar_6',
       'achar_7', 'achar_8', 'achar_9', 'people_id', 'pdate', 'pchar_10',
       'pchar_11', 'pchar_12', 'pchar_13', 'pchar_14', 'pchar_15', 'pchar_16',
       'pchar_17', 'pchar_18', 'pchar_19', 'pchar_20', 'pchar_21', 'pchar_22',
       'pchar_23', 'pchar_24', 'pchar_25', 'pchar_26', 'pchar_27', 'pchar_28',
       'pchar_29', 'pchar_30', 'pchar_31', 'pchar_32', 'pchar_33', 'pchar_34',
       'pchar_35', 'pchar_36', 'pchar_37', 'pchar_38', 'group_1', 'pchar_1',
       'pchar_2', 'pchar_3', 'pchar_4', 'pchar_5', 'pchar_6', 'pchar_7',
       'pchar_8', 'pchar_9', 'outcome_filled', 'outcome_filled_nona',
       'outcome_leak', 'outcome_leak_int', 'achar_10_reduced',
       'adate_dayofweek', 'adate_daynum', 'pdate_daynum',
       'business_days_delta', 'adate_gap', 'outcome_filled_prevday',
       'outcome_filled_nextday', 'people_per_group',

In [37]:
data.adate_dayofweek = data.adate_dayofweek.astype(np.uint8)

data.business_days_delta = data.business_days_delta.astype(np.uint16)

data.adate_daynum = data.adate_daynum.astype(np.int16)
data.pdate_daynum = data.pdate_daynum.astype(np.int16)

data.people_per_group = data.people_per_group.astype(np.int32)
data.people_per_group_adate = data.people_per_group_adate.astype(np.int16)
data.events_per_group_adate = data.events_per_group_adate.astype(np.int16)

In [38]:
data_augments = data[['activity_id', 'achar_10_reduced',
       'adate_dayofweek', 'adate_daynum', 'pdate_daynum',
       'business_days_delta', 'adate_gap', 'outcome_filled_prevday', 'outcome_filled_nextday', 'people_per_group',
       'people_per_group_adate', 'events_per_group_adate']]

In [39]:
for k in data_augments.keys():
    print(k, data_augments[k].dtype, data_augments[k].min(), data_augments[k].max())

activity_id object act1_1 act2_9e+05
achar_10_reduced int16 -1000 9151
adate_dayofweek uint8 0 6
adate_daynum int16 0 410
pdate_daynum int16 -790 410
business_days_delta uint16 0 856
adate_gap float64 1.0 402.0
outcome_filled_prevday float64 0.0 1.0
outcome_filled_nextday float64 0.0 1.0
people_per_group int32 1 77314
people_per_group_adate int16 1 2710
events_per_group_adate int16 1 4249


In [40]:
data_augments.to_pickle('dproc7.pkl')

In [41]:
data_augments.isnull().sum()

activity_id                    0
achar_10_reduced               0
adate_dayofweek                0
adate_daynum                   0
pdate_daynum                   0
business_days_delta            0
adate_gap                  81932
outcome_filled_prevday    436830
outcome_filled_nextday    431203
people_per_group               0
people_per_group_adate         0
events_per_group_adate         0
dtype: int64

In [42]:
data_augments[data_augments.activity_id == 'act2_3889418']

,activity_id,achar_10_reduced,adate_dayofweek,adate_daynum,pdate_daynum,business_days_delta,adate_gap,outcome_filled_prevday,outcome_filled_nextday,people_per_group,people_per_group_adate,events_per_group_adate
1372164,act2_3889418,2251,3,95,74,15,1.0,1.0,1.0,21,3,4
